In [1]:
pip install sentence_transformers

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 645kB 22.4MB/s 
     |████████████████████████████████| 1.0MB 51.5MB/s 
     |████████████████████████████████| 890kB 57.2MB/s 
     |████████████████████████████████| 3.8MB 47.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=dad2bfc0b6eb6912c48983066cfd370dc30d1c266cb698fa4069e8972b99d482
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0443f6910fe395ed85be0f03811bca9dd5ca133c5db64800c13b00049dc9b462
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [0]:
from sentence_transformers import SentenceTransformer
import scipy.spatial as sc
import pandas as pd
import re

In [3]:
embedder = SentenceTransformer('roberta-large-nli-mean-tokens')

100%|██████████| 1.31G/1.31G [00:22<00:00, 57.3MB/s]


In [0]:
df = pd.read_json(r'/content/drive/My Drive/Colab Notebooks/ArticleDataset.json')
df.drop_duplicates(subset = 2, keep='first', inplace= True)
df.reset_index(inplace=True)

In [0]:
sentence = []
tidysentence=[]

In [0]:
for i in range(len(df)):
    st = df[2][i]
    tidysentence.append(st)
    st = re.sub('[^a-zA-z0-9\s]','',st)
    st = '.'.join(st.rsplit('\n'))
    sentence.append(st)

In [7]:
len(sentence)

8361

In [0]:
sentence_embb = embedder.encode(sentence)

In [9]:
query = [str(input('Enter the query:'))]
Closest = int(input("Enter Number"))

Enter the query:Latest Robotics News
Enter Number5


In [0]:
query_embb = embedder.encode(query)

In [0]:
res = []
score = []
for que, que_embb in zip(query,query_embb):
  distance = sc.distance.cdist([que_embb], sentence_embb,'yule')[0]
  result = zip(range(len(distance)), distance)
  result = sorted(result, key=lambda x: x[1])
  
  for ide, leng in result[0:Closest]:
    score.append(1-leng)
    Parag = sentence[ide]
    #print(score)
    #print(Parag)
    res.append(df.iloc[ide])
csv_result = pd.DataFrame(res)


In [0]:
final_res = pd.DataFrame()
final_res["Results"] = csv_result[2]
final_res["URL"] = csv_result[3]
final_res['yule_Scores'] = pd.Series(score, index=final_res.index)
final_res.to_csv(query[0]+'with_yule_similarity.csv')
